<a href="https://colab.research.google.com/github/mayhd3/NSF-REU-2021/blob/main/AMImodels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from keras.utils import np_utils
from scipy.signal import find_peaks

try:
  full
except NameError:
  if os.path.exists('full.pickle'):
    with open('full.pickle','rb') as pk:
      full = pickle.load(pk)

# group dataset by customer
groups = full.groupby(2)
meters = [groups.get_group(group) for group in groups.groups]

# cnn input is a 2D array of semi hourly consumption by week
X0 = [meter.iloc[:,5:].to_numpy() for meter in meters]
y0 = [
  {0: [0,0], 1: [0,1], 2:[1,0], 3:[1,1]}[meter.iloc[0,0]]
    for meter in meters
]

# fnn input is a 1D array of consumption over the year
X1 = [x.flatten() for x in X0]
y1 = [meter[1].to_numpy() for meter in meters]

# rnn input is a time series between consumption minima and maxima
X2 = [np.diff(find_peaks(np.mean(x, axis=1))[0]) for x in X0]
y2 = [y != 0 for y in y0]

def bhwc(twod):
  return np.expand_dims(np.array(twod), axis=-1).astype('float32')

# partition training and testing datasets
def periodic_partition(array, ratio):
  return (
    [array[i] for i in range(len(array)) if i%ratio == 0],
    [array[i] for i in range(len(array)) if i%ratio != 0]
  )

X0_test, X0_train = (bhwc(X) for X in periodic_partition(X0, 3))
y0_test, y0_train = (np.array(y) for y in periodic_partition(y0, 3))

# cnn structure is (conv -> pool) x3 -> dropout -> dense x2
cnn = Sequential()
cnn.add(Conv2D(32, (3,3), input_shape=X0_train.shape[1:], activation='relu'))
#cnn.add(Conv2D(32,(4,4),input_shape=X0_train.shape, activation='relu'))
#cnn.add(MaxPool2D(pool_size=2))
#cnn.add(Conv2D(64,(26,26),activation='relu'))
#cnn.add(MaxPool2D(pool_size=2))
#cnn.add(Conv2D(128,(9,9),activation='relu'))
#cnn.add(MaxPool2D(pool_size=2))
#cnn.add(Dropout(0.0625))
cnn.add(Flatten())
#cnn.add(Dense(128, activation='relu'))
cnn.add(Dense(32, activation='relu'))
cnn.add(Dense(2, activation='softmax'))

cnn.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

cnn.fit(
    X0_train,
    y0_train,
    epochs=52,
    batch_size=38
)
cnn.evaluate(X0_test, y0_test, verbose=2)